In [20]:
!pip install --upgrade pip

#!pip uninstall tensorflow-macos tensorflow-metal -y
!pip install tensorflow tf-slim tensorflow-object-detection-api object_detection
!pip install -q numpy opencv-python-headless matplotlib pandas lxml tensorflow-datasets notebook object_detection Pillow

In [25]:
import os

# Step 1: Change to the main directory where you want to clone models
os.chdir('/Users/qontrol/dev/TF-pigeon-detection/')

# Step 2: Clone the TensorFlow models repository if not done already
!git clone https://github.com/tensorflow/models.git

# Step 3: Update PYTHONPATH to include the 'models' directory
if 'PYTHONPATH' in os.environ:
    os.environ['PYTHONPATH'] += ":/Users/qontrol/dev/TF-pigeon-detection/models"
else:
    os.environ['PYTHONPATH'] = "/Users/qontrol/dev/TF-pigeon-detection/models"

# Step 4: Install the required dependencies
!pip3 install -r models/official/requirements.txt


fatal: destination path 'models' already exists and is not an empty directory.
  Using cached google_api_python_client-2.158.0-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached kaggle-1.6.17-py3-none-any.whl
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached scipy-1.15.1-cp311-cp311-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached tensorflow_model_optimization-0.8.0-py2.py3-none-any.whl.metadata (904 bytes)
  Using cached tf_keras-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached gin_config-0.5.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached tf_slim-1.1.0-py2.py3-none-any.whl.metadata (1.6 kB)
  Using cached Cython-3.0.11-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached pycocotools-2.0.8-cp311-cp311-macosx_10_9_universal2.whl.metadata (1.1 kB)
  Using cached seqeval-1.2.2-py3-none-

In [22]:
!brew install protobuf


==> Downloading https://formulae.brew.sh/api/formula.jws.json
######################################################################### 100.0%
==> Downloading https://formulae.brew.sh/api/cask.jws.json
######################################################################### 100.0%
To reinstall 29.3, run:
  brew reinstall protobuf


In [24]:
import os
os.chdir('/Users/qontrol/dev/TF-pigeon-detection/models/research/')
!protoc --proto_path=/Users/qontrol/dev/TF-pigeon-detection/models/research/ \
        /Users/qontrol/dev/TF-pigeon-detection/models/research/object_detection/protos/*.proto \
        --python_out=/Users/qontrol/dev/TF-pigeon-detection/models/research/



In [25]:
!python /Users/qontrol/dev/TF-pigeon-detection/models/research/object_detection/builders/model_builder_tf2_test.py


Traceback (most recent call last):
  File "/Users/qontrol/dev/TF-pigeon-detection/models/research/object_detection/builders/model_builder_tf2_test.py", line 24, in <module>
    from object_detection.builders import model_builder
  File "/Users/qontrol/dev/TF-pigeon-detection/pigeon_env/lib/python3.11/site-packages/object_detection/builders/model_builder.py", line 20, in <module>
    from object_detection.builders import anchor_generator_builder
  File "/Users/qontrol/dev/TF-pigeon-detection/pigeon_env/lib/python3.11/site-packages/object_detection/builders/anchor_generator_builder.py", line 21, in <module>
    from object_detection.protos import anchor_generator_pb2
  File "/Users/qontrol/dev/TF-pigeon-detection/pigeon_env/lib/python3.11/site-packages/object_detection/protos/anchor_generator_pb2.py", line 16, in <module>
    from object_detection.protos import grid_anchor_generator_pb2 as object__detection_dot_protos_dot_grid__anchor__generator__pb2
  File "/Users/qontrol/dev/TF-pigeon-

In [32]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

# Function to parse the XML file and extract the relevant data
def parse_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    filename = root.find('filename').text
    objects = root.findall('object')

    data = []
    for obj in objects:
        # Get the class name and bounding box coordinates
        class_name = obj.find('name').text
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)
        data.append([filename, xmin, ymin, xmax, ymax, class_name])
    
    return data

# Function to convert all XML files in a directory to a CSV file
def convert_xml_to_csv(xml_dir, output_csv):
    data = []
    for xml_file in os.listdir(xml_dir):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(xml_dir, xml_file)
            data.extend(parse_xml(xml_path))
    
    # Create a DataFrame and save it as a CSV
    df = pd.DataFrame(data, columns=['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'class'])
    df.to_csv(output_csv, index=False)
    print(f'CSV file saved at {output_csv}')

# Path to your XML annotations
xml_dir = '/Users/qontrol/dev/TF-pigeon-detection/data/images/train'

# Output CSV file path
output_csv = '/Users/qontrol/dev/TF-pigeon-detection/data/images/train/train_annotations.csv'

# Convert XML to CSV
convert_xml_to_csv(xml_dir, output_csv)


CSV file saved at /Users/qontrol/dev/TF-pigeon-detection/data/images/train/train_annotations.csv


In [10]:
import os
import tensorflow as tf
from object_detection.utils import dataset_util
import pandas as pd
from PIL import Image
import numpy as np
from collections import namedtuple

# Helper function to create the example for the TFRecord file
def create_tf_example(group, path, class_map, resize_width, resize_height):
    # Get the image dimensions and resize
    image_path = os.path.join(path, group['filename'].iloc[0])
    with Image.open(image_path) as img:
        # Resize image
        img = img.resize((resize_width, resize_height))
        width, height = img.size

    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_jpg = fid.read()

    image_format = b'jpg'

    # Convert the bounding box coordinates to float32 (handle multiple rows)
    xmins = [float(group['xmin'].iloc[0]) / width]
    xmaxs = [float(group['xmax'].iloc[0]) / width]
    ymins = [float(group['ymin'].iloc[0]) / height]
    ymaxs = [float(group['ymax'].iloc[0]) / height]

    # Map the class name to an ID using class_map
    class_id = class_map[group['class'].iloc[0]]

    # Create the example
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(group['filename'].iloc[0].encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(group['filename'].iloc[0].encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature([group['class'].iloc[0].encode('utf8')]),
        'image/object/class/label': dataset_util.int64_list_feature([class_id])
    }))
    return tf_example

# Main function to convert CSV to TFRecord
def create_tf_record(csv_input_path, image_dir, output_path, resize_width, resize_height):
    writer = tf.io.TFRecordWriter(output_path)
    
    # Read the CSV file
    examples = pd.read_csv(csv_input_path)

    # Create a mapping from class names to class ids
    class_names = examples['class'].unique()
    class_map = {class_name: idx + 1 for idx, class_name in enumerate(class_names)}

    # Group examples by filename
    grouped = examples.groupby('filename')

    # Process each grouped image
    for filename, group in grouped:
        print(f"Processing {filename}...")  # Debug print to show progress on which file is being processed
        
        # Sort by class to ensure consistent ordering
        group = group.sort_values('class')

        # Create the TFRecord example for the group
        tf_example = create_tf_example(group, image_dir, class_map, resize_width, resize_height)
        
        if tf_example:  # Only write to TFRecord if the example was created successfully
            writer.write(tf_example.SerializeToString())

    writer.close()

# Convert your CSV to TFRecord
csv_input_path = '/Users/qontrol/dev/TF-pigeon-detection/data/images/train/train_annotations.csv'
image_dir = '/Users/qontrol/dev/TF-pigeon-detection/data/images/train'
output_path = '/Users/qontrol/dev/TF-pigeon-detection/data/tfrecord/train.record'

# Set the desired resize dimensions
resize_width = 300  # Adjust width as needed
resize_height = 300  # Adjust height as needed

create_tf_record(csv_input_path, image_dir, output_path, resize_width, resize_height)


Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_12348.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_12357.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_12384.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_12595.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_12656.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_12695.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_31734.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_31734_augmented_32330.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_31734_augmented_62779.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_31734_augmented_85657.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_31771.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_31771_augmented_32330.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.47[R]_31771_augmented_62779.jpg...
Processing 2019-08-17-18hour-mp4-18.26.34-19.01.4

In [13]:
import os
import tensorflow as tf

# Path where you want to store the model
model_dir = '/Users/qontrol/dev/TF-pigeon-detection/models'
model_name = 'ssd_mobilenet_v2_320x320_coco17_tpu-8'
model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz'

# Create directory if it doesn't exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Download the model tar.gz file to the model directory
print(f"Downloading {model_name} to {model_dir}...")
tf.keras.utils.get_file(
    fname=f'{model_name}.tar.gz', 
    origin=model_url, 
    cache_dir=model_dir, 
    untar=True
)

print("Model download complete.")


46042990/46042990 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step 
Model download complete.


In [14]:
pipeline_config = """
# Configuration for fine-tuning SSD MobileNet V2 model on custom dataset

model {
  ssd {
    num_classes: 1  # Adjust this to the number of classes in your dataset
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 320
        max_dimension: 320
      }
    }
    feature_extractor {
      type: 'ssd_mobilenet_v2'
      depth_multiplier: 1.0
    }
    matcher {
      type: "argmax"
    }
    box_predictor {
      type: "convolutional"
    }
  }
}

train_config {
  batch_size: 11
  # Total number of training steps
  num_steps: 40000
  # Number of evaluation steps
  num_eval_steps: 500
  fine_tune_checkpoint: "/Users/qontrol/dev/TF-pigeon-detection/models/ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint"
  # Path to your label map file
  label_map_path: "/Users/qontrol/dev/TF-pigeon-detection/data/label_map.pbtxt"
  # Paths to your training and validation data in TFRecord format
  train_input_path: "/Users/qontrol/dev/TF-pigeon-detection/data/tfrecord/train.record"
  eval_input_path: "/Users/qontrol/dev/TF-pigeon-detection/data/tfrecord/val.record"
  
  # Optimization settings
  optimizer {
    momentum_optimizer {
      learning_rate {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.004
          decay_steps: 800000
          decay_factor: 0.95
        }
      }
      momentum: 0.9
    }
    use_moving_average: false
  }

  # Augmentation options
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}

# Specify paths for the fine-tune checkpoint, label map, and TFRecord data
train_input_path: "/Users/qontrol/dev/TF-pigeon-detection/data/tfrecord/train.record"
eval_input_path: "/Users/qontrol/dev/TF-pigeon-detection/data/tfrecord/val.record"
fine_tune_checkpoint: "/Users/qontrol/dev/TF-pigeon-detection/models/ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint"
label_map_path: "/Users/qontrol/dev/TF-pigeon-detection/data/label_map.pbtxt"

# Additional training configurations
train_config {
  batch_size: 11  # Set batch size based on your system's memory
  num_steps: 40000
  # Optional: Number of steps for evaluation
  num_eval_steps: 500
}
"""

# Write the config to a file
pipeline_config_path = '/Users/qontrol/dev/TF-pigeon-detection/data/pipeline.config'

with open(pipeline_config_path, 'w') as f:
    f.write(pipeline_config)

print(f"Pipeline config saved to {pipeline_config_path}")


Pipeline config saved to /Users/qontrol/dev/TF-pigeon-detection/data/pipeline.config


In [15]:
label_map_content = """
item {
  id: 1
  name: 'Pigeon'
}
item {
  id: 2
  name: 'Other'
}
"""

# Path to save label_map.pbtxt
label_map_path = '/Users/qontrol/dev/TF-pigeon-detection/data/label_map.pbtxt'

# Save the label map as a text file
with open(label_map_path, 'w') as f:
    f.write(label_map_content)

print(f"label_map.pbtxt saved at {label_map_path}")


label_map.pbtxt saved at /Users/qontrol/dev/TF-pigeon-detection/data/label_map.pbtxt


In [22]:
!python /Users/qontrol/dev/TF-pigeon-detection/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=/Users/qontrol/dev/TF-pigeon-detection/data/pipeline.config \
    --model_dir=/Users/qontrol/dev/TF-pigeon-detection/models/ssd_mobilenet_v2_320x320_coco17_tpu-8 \
    --alsologtostderr


Traceback (most recent call last):
  File "/Users/qontrol/dev/TF-pigeon-detection/models/research/object_detection/model_main_tf2.py", line 31, in <module>
    from object_detection import model_lib_v2
ImportError: cannot import name 'model_lib_v2' from 'object_detection' (/Users/qontrol/dev/TF-pigeon-detection/pigeon_env/lib/python3.11/site-packages/object_detection/__init__.py)


In [9]:
from object_detection import model_lib_v2

ImportError: cannot import name 'runtime_version' from 'google.protobuf' (/Users/qontrol/dev/TF-pigeon-detection/pigeon_env/lib/python3.11/site-packages/google/protobuf/__init__.py)